 #### This file contains all the method and variations that were tried in order to solve the problem with dimensio 5 & 10


### Warning: DEAD CODE  
### All the code inside this file doesn't work if compiled "as is"                                                                

In [ ]:
def uniform_crossover_multiple_ind(W, p):
    k = len(W)
    l = len(W[0])
    
    # Initialization of v
    v = [0] * l
    
    # Crossover Operation
    for i in range(l):
        #  Check probability
        if p >= randint(0,1):
            # Load v with ith elements from each vector in W
            v = [W[j][i] for j in range(k)]
            # Randomly Shuffle v
            shuffle(v)
            
            #Put back Operation
            for j in range(k):
                W[j][i] = v[j]

    #Return modified set of vectors W
    return W

In [ ]:
def apply_fitness_penalty(pop,off):
    # apply a penalty to the fitness of the individuals
    # this is done to avoid that all individuals have the same fitness
    radius=10
    alpha=1.5 #alpha>0
    beta=2.5  #beta>1
    pop=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)
    off_ind=pop.index(off)
    if off_ind<radius:
        min_ind=0
    else:
        min_ind=off_ind-radius
    if off_ind+radius>len(pop)-1:
        max_ind=len(pop)
    else:
        max_ind=off_ind+radius
    for i in range(min_ind,max_ind):
       dist=hamming_distance(pop[i].get_genome(),off.get_genome())
       if dist<10:
            pen=(1-(pop[i].get_fitness()/radius)**alpha)
            new_fitness=(pop[i].get_fitness()**beta)/pen
    return pop

### Entropy


In [ ]:
def compute_probabilities(pop):
    concat=''
    for i in range(len(pop)):
        concat+=''.join(map(str,pop[i].get_genome()))
    p=[]
    for i in range(len(pop)):
        to_compare=concat[i*GENOME_LENGHT:(i+1)*GENOME_LENGHT] #prendo intervallo di 1000
        count=0
        for j in range(i,len(pop)):
            to_compare2=concat[j*GENOME_LENGHT:(j+1)*GENOME_LENGHT]
            if to_compare==to_compare2:
                count+=1
        p.append(count/len(pop))
    return p
def compute_pop_entropy(pop):
    # compute population entropy
    # to avod numerical problems, we add a small value (1e-10) to the probability
    prob=compute_probabilities(pop)
    entropy = -sum([p * np.log2(p+1e-10) for p in prob ])
    return entropy/np.log2(GENOME_LENGHT)

def entropy_stocastic_selection(pop):
    max_entropy = -1
    candidates = sample(range(len(pop)), k=30)
    candidates = [pop[i] for i in candidates]
    candidates=sorted(candidates, key=lambda i: i.get_fitness(), reverse=True)[:3]
    for i in range(len(candidates)):
        for j in range(i + 1, len(candidates)):
            copy_pop = copy(pop)
            new_ind=two_cut_xover(candidates[i], candidates[j]) # crossover
            copy_pop.append(new_ind)
            new_etr=compute_pop_entropy(copy_pop)

            if new_etr > max_entropy:
                max_entropy = new_etr
                best_ind = new_ind
    return best_ind